In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, datasets
from torchvision.transforms import v2 as transforms
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score

In [ ]:
torch.cuda.is_available()

In [4]:
transform_test = transforms.Compose([
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToImage(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3)),
])

In [5]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=3)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=3)


In [6]:
class ResNet18_CIFAR10(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18_CIFAR10, self).__init__()
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()

        self.model.fc = nn.Sequential(
            nn.Dropout(p=0.3),
            nn.Linear(self.model.fc.in_features, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNet18_CIFAR10()
model = model.to(device)

In [8]:
criterion = nn.CrossEntropyLoss()

# optimizer = optim.Adam(model.parameters(), lr=0.015, weight_decay=1e-3)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.8, weight_decay=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.2)

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_labels = []
    all_predictions = []

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

    scheduler.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")
    print(f"Precision: {precision_score(all_labels, all_predictions, average='macro', zero_division=1):.2f}")
    print(f"Recall: {recall_score(all_labels, all_predictions, average='macro', zero_division=1):.2f}")


# import IPython
# display(IPython.display.Audio(url="https://static.sfdict.com/audio/C07/C0702600.mp3", autoplay=True))

In [ ]:
model.eval()

total = 0

correct = 0
test_loss = 0.0
all_labels = []
all_predictions = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        #outputs = model(F.interpolate(images, size=(224, 224), mode="bilinear", align_corners=False))
        outputs = model(images)

        _, predicted = torch.max(outputs, 1)

        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, labels)
        test_loss += loss.item() * labels.size(0)

        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
test_loss = test_loss/total
precision = precision_score(all_labels, all_predictions, average='macro', zero_division=1)
recall = recall_score(all_labels, all_predictions, average='macro', zero_division=1)

print(f"Test Accuracy: {accuracy:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

In [ ]:
def imshow(img, std, mean, t_label, p_label):
    mean = torch.tensor(mean).view(3, 1, 1).to(device)
    std = torch.tensor(std).view(3, 1, 1).to(device)


    img = img * std + mean
    img = img.clamp(0, 1)
    img = img.permute(1, 2, 0)
    npimg = img.cpu().numpy()

    plt.imshow(npimg)
    plt.axis("off")
    plt.show()
    print(f"True label: {labels[t_label]}")
    print(f"Predicted label: {labels[p_label]}")

cnt = 0

labels = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
for images, labels_num in test_loader:
    images, labels_num = images.to(device), labels_num.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    cnt += 1

    imshow(images[0], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], t_label = labels_num[0].item(), p_label = predicted[0].item())
    if cnt == 5:
      break

In [16]:
torch.save(model.state_dict(), 'model1.pth')